In [1]:
debug = True
def Dprint(data, debug=debug):
    if debug:
        print(data)

# tools.reverse.py

In [2]:
def reverse_string(input_string):
    """
    Reverse the given string.

    Parameters:
    input_string (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    Dprint(f"DEBUG: input_string for Reverse: {input_string}")
    # Reverse the string using slicing
    reversed_string = input_string[::-1]

    reversed_string = f"The reversed string is: {reversed_string}\n\n.Executed using the reverse_string function."
    # print (f"DEBUG: reversed_string: {reversed_string}")
    return reversed_string

# tools.basic_calculator.py

In [3]:
import operator
import json

def basic_calculator(input_str):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a dictionary with keys 'num1', 'num2', and 'operation'.
                     Example: '{"num1": 5, "num2": 3, "operation": "add"}' or "{'num1': 67869, 'num2': 9030393, 'operation': 'divide'}"

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """
    # Clean and parse the input string
    try:
        # Replace single quotes with double quotes
        Dprint(f"DEBUG: basic calculator input_str: {input_str}")
        input_str_clean = input_str.replace("'", "\"")
        # Remove any extraneous characters such as trailing quotes
        input_str_clean = input_str_clean.strip().strip("\"")
        
        input_dict = json.loads(input_str_clean)
        num1 = input_dict['num1']
        num2 = input_dict['num2']
        operation = input_dict['operation']
    except (json.JSONDecodeError, KeyError) as e:
        return str(e), "Invalid input format. Please provide a valid JSON string."

    # Define the supported operations
    operations = {
        'add': operator.add,
        'subtract': operator.sub,
        'multiply': operator.mul,
        'divide': operator.truediv,
        'floor_divide': operator.floordiv,
        'modulus': operator.mod,
        'power': operator.pow,
        'lt': operator.lt,
        'le': operator.le,
        'eq': operator.eq,
        'ne': operator.ne,
        'ge': operator.ge,
        'gt': operator.gt
    }
    
    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2) # operations[operation] is the function to be called
            result_formatted = f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."


# utils.get_keys

In [4]:
import os
import yaml 

def load_config(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
        for key, value in config.items():
            os.environ[key] = value

In [5]:
config_path = os.path.join(os.getcwd(), 'config.yaml')
print(config_path)
load_config(config_path)
os.getenv('TEST')

/Users/ajeet/Data/Development/Learning-Research-Development/CustomAgents/config.yaml


'test'

# prompts.prompts.py

In [12]:
agent_system_prompt_template = """
You are an agent with access to a toolbox. Given a user query, 
you will determine which tool, if any, is best suited to answer the query. 
You will generate the following JSON response:

"tool_choice": "name_of_the_tool",
"tool_input": "inputs_to_the_tool"

- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox 
                or "no tool" if you do not need to use a tool.
- `tool_input`: The specific inputs required for the selected tool. 
                If no tool, just provide a response to the query.

Here is a list of your tools along with their descriptions:
{tool_descriptions}

Please make a decision based on the provided user query and the available tools.
"""

# models.ollama_models.py

In [17]:
import requests
import json
import os

# Load configuration

class OllamaModel:
    def __init__(self, model, system_prompt, temperature=0, stop=None):
        """
        Initializes the OllamaModel with the given parameters.

        Parameters:
        model (str): The name of the model to use.
        system_prompt (str): The system prompt to use.
        temperature (float): The temperature setting for the model.
        stop (str): The stop token for the model.
        """
        self.model_endpoint = "http://localhost:11434/api/generate"
        self.temperature = temperature
        self.model = model
        self.system_prompt = system_prompt
        self.headers = {"Content-Type": "application/json"}
        self.stop = stop

    def generate_text(self, prompt):
        """
        Generates a response from the Ollama model based on the provided prompt.

        Parameters:
        prompt (str): The user query to generate a response for.

        Returns:
        dict: The response from the model as a dictionary.
        """
        payload = {
            "model": self.model,
            "format": "json",
            "prompt": prompt,
            "system": self.system_prompt,
            "stream": False,
            "temperature": self.temperature,
            "stop": self.stop
        }

        try:
            request_response = requests.post(
                self.model_endpoint, 
                headers=self.headers, 
                data=json.dumps(payload)
            )
            
            print("REQUEST RESPONSE", request_response)
            request_response_json = request_response.json()
            response = request_response_json['response']
            response_dict = json.loads(response)

            print(f"\n\nResponse from Ollama model: {response_dict}")

            return response_dict
        except requests.RequestException as e:
            response = {"error": f"Error in invoking model! {str(e)}"}
            return response


# models.openai_models.py

In [22]:
import requests
import json
import os
# from utils.get_keys import load_config

# config_path = os.path.join(os.path.dirname(__file__), '..', 'configs', 'config.yaml')
config_path = os.path.join(os.getcwd(), 'config.yaml')
load_config(config_path)

class OpenAIModel:
    def __init__(self, model, system_prompt, temperature):
        self.model_endpoint = 'https://api.openai.com/v1/chat/completions'
        self.temperature = temperature
        self.model = model
        self.system_prompt = system_prompt
        load_config(config_path)
        self.api_key = os.getenv('OPENAI_API_KEY')
        self.headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.api_key}'
        }


    def generate_text(self, prompt):

        payload = {
                    "model": self.model,
                    "response_format": {"type": "json_object"},
                    "messages": [
                        {
                            "role": "system",
                            "content": self.system_prompt
                        },
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    "stream": False,
                    "temperature": self.temperature,
                }
        Dprint(f"DEBUG: openai payload: {payload}")
        
        response_dict = requests.post(self.model_endpoint, headers=self.headers, data=json.dumps(payload))
        response_json = response_dict.json()
        response = json.loads(response_json['choices'][0]['message']['content'])

        print(F"\n\nResponse from OpenAI model: {response}")

        return response

# toolbox.toolbox.py

In [23]:
class ToolBox:
    def __init__(self):
        self.tools_dict = {}

    def store(self, functions_list):
        """
        Stores the literal name and docstring of each function in the list.

        Parameters:
        functions_list (list): List of function objects to store.

        Returns:
        dict: Dictionary with function names as keys and their docstrings as values.
        """
        for func in functions_list:
            self.tools_dict[func.__name__] = func.__doc__
        Dprint(f"DEBUG: tools_dict: {self.tools_dict}")
        return self.tools_dict

    def tools(self):
        """
        Returns the dictionary created in store as a text string.

        Returns:
        str: Dictionary of stored functions and their docstrings as a text string.
        """
        tools_str = ""
        for name, doc in self.tools_dict.items():
            tools_str += f"{name}: \"{doc}\"\n"
        Dprint(f"DEBUG: tools_str: {tools_str}")
        return tools_str.strip()

# agents.Agent

In [24]:
from termcolor import colored
class Agent:
    def __init__(self, tools, model_service, model_name, stop=None):
        """
        Initializes the agent with a list of tools and a model.

        Parameters:
        tools (list): List of tool functions.
        model_service (class): The model service class with a generate_text method.
        model_name (str): The name of the model to use.
        """
        self.tools = tools # List of tool functions
        self.model_service = model_service
        self.model_name = model_name
        self.stop = stop

    def prepare_tools(self):
        """
        Stores the tools in the toolbox and returns their descriptions.

        Returns:
        str: Descriptions of the tools stored in the toolbox.
        """
        toolbox = ToolBox()
        toolbox.store(self.tools)
        tool_descriptions = toolbox.tools()
        Dprint(f"DEBUG: tool_descriptions: {tool_descriptions}")
        return tool_descriptions

    def think(self, prompt):
        """
        Runs the generate_text method on the model using the system prompt template and tool descriptions.

        Parameters:
        prompt (str): The user query to generate a response for.

        Returns:
        dict: The response from the model as a dictionary.
        """
        tool_descriptions = self.prepare_tools()
        agent_system_prompt = agent_system_prompt_template.format(tool_descriptions=tool_descriptions)
        Dprint(f"DEBUG: agent_system_prompt: {agent_system_prompt}")

        # Create an instance of the model service with the system prompt

        if self.model_service == OllamaModel:
            model_instance = self.model_service(
                model=self.model_name,
                system_prompt=agent_system_prompt,
                temperature=0,
                stop=self.stop
            )
        else:
            model_instance = self.model_service(
                model=self.model_name,
                system_prompt=agent_system_prompt,
                temperature=0
            )

        # Generate and return the response dictionary
        Dprint(f"DEBUG: prompt: {prompt}")
        agent_response_dict = model_instance.generate_text(prompt)
        Dprint(f"DEBUG: agent_response_dict: {agent_response_dict}")
        return agent_response_dict

    def work(self, prompt):
        """
        Parses the dictionary returned from think and executes the appropriate tool.

        Parameters:
        prompt (str): The user query to generate a response for.

        Returns:
        The response from executing the appropriate tool or the tool_input if no matching tool is found.
        """
        agent_response_dict = self.think(prompt)
        Dprint(f"DEBUG: work agent_response_dict: {agent_response_dict}")
        tool_choice = agent_response_dict.get("tool_choice")
        Dprint(f"DEBUG: work tool_choice: {tool_choice}")
        tool_input = agent_response_dict.get("tool_input")
        Dprint(f"DEBUG: work tool_input: {tool_input}")

        for tool in self.tools:
            if tool.__name__ == tool_choice:
                response = tool(tool_input)

                print(colored(response, 'cyan'))
                return
                # return tool(tool_input)

        print(colored(tool_input, 'cyan'))
        
        return


# Run

In [29]:
if __name__ == "__main__":

    tools = [basic_calculator, reverse_string]


    # Uncoment below to run with OpenAI
    model_name = 'gpt-4o'
    stop = None
    model_service = OpenAIModel


    # Uncomment below to run with Ollama
    # model_service = OllamaModel
    # model_name = 'llama3:instruct'
    # stop = "<|eot_id|>"

    agent = Agent(tools=tools, model_service=model_service, model_name=model_name, stop=stop)

    while True:
        prompt = input("Ask me anything: ")
        if prompt.lower() == "exit":
            break
    
        agent.work(prompt)

DEBUG: tools_dict: {'basic_calculator': '\n    Perform a numeric operation on two numbers based on the input string.\n\n    Parameters:\n    input_str (str): A JSON string representing a dictionary with keys \'num1\', \'num2\', and \'operation\'.\n                     Example: \'{"num1": 5, "num2": 3, "operation": "add"}\' or "{\'num1\': 67869, \'num2\': 9030393, \'operation\': \'divide\'}"\n\n    Returns:\n    str: The formatted result of the operation.\n\n    Raises:\n    Exception: If an error occurs during the operation (e.g., division by zero).\n    ValueError: If an unsupported operation is requested or input is invalid.\n    ', 'reverse_string': '\n    Reverse the given string.\n\n    Parameters:\n    input_string (str): The string to be reversed.\n\n    Returns:\n    str: The reversed string.\n    '}
DEBUG: tools_str: basic_calculator: "
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a d

In [31]:
print('''{'model': 'gpt-4o', 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'system', 'content': '\nYou are an agent with access to a toolbox. Given a user query, \nyou will determine which tool, if any, is best suited to answer the query. \nYou will generate the following JSON response:\n\n"tool_choice": "name_of_the_tool",\n"tool_input": "inputs_to_the_tool"\n\n- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox \n                or "no tool" if you do not need to use a tool.\n- `tool_input`: The specific inputs required for the selected tool. \n                If no tool, just provide a response to the query.\n\nHere is a list of your tools along with their descriptions:\nbasic_calculator: "\n    Perform a numeric operation on two numbers based on the input string.\n\n    Parameters:\n    input_str (str): A JSON string representing a dictionary with keys \'num1\', \'num2\', and \'operation\'.\n                     Example: \'{"num1": 5, "num2": 3, "operation": "add"}\' or "{\'num1\': 67869, \'num2\': 9030393, \'operation\': \'divide\'}"\n\n    Returns:\n    str: The formatted result of the operation.\n\n    Raises:\n    Exception: If an error occurs during the operation (e.g., division by zero).\n    ValueError: If an unsupported operation is requested or input is invalid.\n    "\nreverse_string: "\n    Reverse the given string.\n\n    Parameters:\n    input_string (str): The string to be reversed.\n\n    Returns:\n    str: The reversed string.\n    "\n\nPlease make a decision based on the provided user query and the available tools.\n'}, {'role': 'user', 'content': '23313/435+234234?'}], 'stream': False, 'temperature': 0}''')

{'model': 'gpt-4o', 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'system', 'content': '
You are an agent with access to a toolbox. Given a user query, 
you will determine which tool, if any, is best suited to answer the query. 
You will generate the following JSON response:

"tool_choice": "name_of_the_tool",
"tool_input": "inputs_to_the_tool"

- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox 
                or "no tool" if you do not need to use a tool.
- `tool_input`: The specific inputs required for the selected tool. 
                If no tool, just provide a response to the query.

Here is a list of your tools along with their descriptions:
basic_calculator: "
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a dictionary with keys 'num1', 'num2', and 'operation'.
                     Example: '{"num1": 5, "num2": 3, "operation":